In [49]:
%run __init__.py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
from itertools import combinations
from tools import *
from before_stack import *
from conversion import *
from aper_phot import *
from cr_to_flux import *
mon = 'dec'

In [50]:
#untar_obs('Borisov_raw_dec')
'''
make_obs_log(horizon_id=90004424,
             folder_name='Borisov_raw_dec',
             map_type='sk',
             output_name='dec_obs-log_Borisov.txt')
'''

"\nmake_obs_log(horizon_id=90004424,\n             folder_name='Borisov_raw_dec',\n             map_type='sk',\n             output_name='dec_obs-log_Borisov.txt')\n"

In [51]:
# stack image
'''
stack_image(obs_log_name='dec_obs-log_Borisov_part.txt',
            filt='uvv', size=(1000,1000),
            output_name='dec_stack_part')

stack_image(obs_log_name='dec_obs-log_Borisov_part.txt',
            filt='uw1', size=(1000,1000),
            output_name='dec_stack_part')
'''

"\nstack_image(obs_log_name='dec_obs-log_Borisov_part.txt',\n            filt='uvv', size=(1000,1000),\n            output_name='dec_stack_part')\n\nstack_image(obs_log_name='dec_obs-log_Borisov_part.txt',\n            filt='uw1', size=(1000,1000),\n            output_name='dec_stack_part')\n"

In [52]:
# aperture photometry
# src_method = 'count', 'median'
# bg_method = 'circle_mean', 'circle_median', 'donut_mean', 'donut_median', 'multi_mean'
src_center = (1000, 1000)
if mon == 'sep':
    img_name_uw1 = 'sep_stack_uw1.fits'
    img_name_v = 'sep_stack_part_uvv.fits'
    src_r = 15
    bg_reg_name_uw1 = 'sep_bg_uw1_new.reg'
    bg_reg_name_v = 'sep_bg_uvv_new.reg'
if mon == 'nov':
    img_name_uw1 = 'nov_stack_uw1.fits'
    img_name_v = 'nov_stack_uvv.fits'
    src_r = 50
    bg_reg_name_uw1 = 'nov_bg_uw1_new.reg'
    bg_reg_name_v = 'nov_bg_uvv_new.reg'
if mon == 'dec':
    img_name_uw1 = 'dec_stack_uw1.fits'
    img_name_v = 'dec_stack_uvv.fits'
    src_r = 40
    bg_reg_name_uw1 = 'dec_bg_uw1_new.reg'
    bg_reg_name_v = 'dec_bg_uvv_new.reg'
# uw1

bg_uw1 = load_reg_list(bg_reg_name_uw1)
bg_x_uw1 = bg_uw1[1]
bg_y_uw1 = bg_uw1[0]
bg_r_uw1 = bg_uw1[2]
bg_center_uw1 = list(zip(bg_x_uw1, bg_y_uw1))
bg_v = load_reg_list(bg_reg_name_v)
bg_x_v = bg_v[1]
bg_y_v = bg_v[0]
bg_r_v = bg_v[2]
bg_center_v = list(zip(bg_x_v, bg_y_v))
    
result_uw1 = aper_phot(img_name_uw1, 'uw1', 
                       #src_center_uw1, src_r_uw1, 
                       src_center, src_r,
                       bg_center_uw1, bg_r_uw1,
                       'count', 'multi_mean')


cr_uw1, cr_uw1_err = result_uw1[0]
snr_uw1 = result_uw1[1]
mag_uw1, mag_uw1_err = result_uw1[2]
bg_cr_uw1, bg_cr_uw1_err = result_uw1[3]

print('UW1:\n'
      +'COUNT RATE (cts/s): '+str(cr_uw1)+' +/- '+str(cr_uw1_err)+'\n'
      +'SNR: '+str(snr_uw1)+'\n'
      +'MAGNITUDE (mag): '+str(mag_uw1)+' +/- '+str(mag_uw1_err)+'\n'
      +'BACKGROUND CR (cts/s/arcsec2): '+str(bg_cr_uw1)+' +/- '+str(bg_cr_uw1_err)+'\n')
    
result_v = aper_phot(img_name_v, 'v',
                     #src_center_v, src_r_v, 
                     src_center, src_r,
                     bg_center_v, bg_r_v,
                     'count', 'multi_mean')

cr_v, cr_v_err = result_v[0]
snr_v = result_v[1]
mag_v, mag_v_err = result_v[2]
bg_cr_v, bg_cr_v_err = result_v[3]

print('V:\n'
      +'COUNT RATE (cts/s): '+str(cr_v)+' +/- '+str(cr_v_err)+'\n'
      +'SNR: '+str(snr_v)+'\n'
      +'MAGNITUDE (mag): '+str(mag_v)+' +/- '+str(mag_v_err)+'\n'
      +'BACKGROUND CR (cts/s/arcsec2): '+str(bg_cr_v)+' +/- '+str(bg_cr_v_err)+'\n')


UW1:
COUNT RATE (cts/s): 1.4038705657669623 +/- 0.05635475177080336
SNR: 24.91130777181932
MAGNITUDE (mag): 17.12168232874282 +/- 0.05291097464972382
BACKGROUND CR (cts/s/arcsec2): 0.0023460622695572987 +/- 4.803332714463078e-06

V:
COUNT RATE (cts/s): 8.117312354063658 +/- 0.6566104181383055
SNR: 12.362448310032548
MAGNITUDE (mag): 15.616469355164707 +/- 0.08878226198647621
BACKGROUND CR (cts/s/arcsec2): 0.021037301349640297 +/- 7.375360904696688e-05



In [53]:
# calculate OH
spec_name_sun = 'sun_1A.txt'
spec_name_OH = '2019-07-15_emission_models_OH.txt'
spec_name_sum = 'cont_emis_sum.txt'

# method = 'cr', 'flux_cf', 'flux_my_cf', 'flux_cr'
flux, flux_err = get_OH('flux_cr', 
                        spec_name_sun, 
                        spec_name_OH, 
                        spec_name_sum,
                        cr_uw1, cr_uw1_err,
                        cr_v, cr_v_err,
                        0)
print('flux of OH: '+str(flux)+' +/- '+str(flux_err))


beta: 0.09276191501510327
cr to flux: 1.2750906353215913e-12
flux of uw1 (reflection): 7.227546745101541e-12 +/- 5.846371660244792e-13
flux of v: 1.9604669898410466e-12 +/- 1.5858242159443944e-13
flux of OH: 8.299477308689231e-13 +/- 1.0580714144152021e-13


In [54]:
obs_log_name = mon+'_obs-log_Borisov.txt'
wvm_name = mon+'_wvm.txt'
    
num, num_err = flux2num(flux, flux_err, 
                        'fluorescenceOH.txt', 
                        obs_log_name, 
                        method='both_ends', 
                        horizon_id=90004424)
print('number of molecules: '
      + str(num) + ' +/- ' + str(num_err))

q, q_err = num2q(num, num_err, 
                 wvm_name, aperture=src_r)
print('production rate (mol s-1): '
      + str(q) + ' +/- ' + str(q_err))

start time: 2019-12-01T03:28:18.000
end time: 2019-12-01T21:16:55.000
flux to num: 2.1459070334570388e+43
mid-time r: 2.012781827365 (AU)
mid-time rv: -2.9978846 (km/s)
mid-time delta: 2.0442848638869853 (AU)
mid-time g factor (for 3 total lines): 5.47687225116026e-16 (erg s-1 mol-1)
number of molecules: 1.7809906730733316e+31 +/- 2.2705228900934193e+30
num to q: 4.641099672325722e-05
production rate (mol s-1): 8.265755229215806e+26 +/- 1.0537723041220619e+26


In [55]:
# Effective visual bond albedo = 0.05
# Effective thermal albedo = 1
if mon == 'sep':
    z = 6.239E+16
elif mon == 'nov':
    z = 8.747E+16
elif mon == 'dec':
    z = 1.013E+17
active_area = q/z
active_area_err = q_err/z
r = np.sqrt(active_area/(4*np.pi))
r_err = active_area_err/(4*np.sqrt(np.pi*active_area))

print('active area(km2): '+str(active_area/1e10)+' +/- '+str(active_area_err/1e10))
print('radiu(km): '+str(r/1e5)+' +/- '+str(r_err/1e5))

active area(km2): 0.8159679397054102 +/- 0.10402490662606732
radiu(km): 0.2548188876522163 +/- 0.01624298560317834


In [56]:
def q_from_v(v, v_err):
    log_q = 30.675 - 0.2453*v
    log_q_err_0 = error_prop('mul', 
                             0.2453, 0.0013,
                             v, v_err)
    log_q_err = error_prop('sub',
                           30.675, 0.007,
                           0.2453*v, log_q_err_0)
    q = np.power(10, log_q)
    q_err = np.power(10, log_q)*np.log(10.)*log_q_err
    return q, q_err

q_from_v(result_v[2][0], result_v[2][1])

(6.98682824366647e+26, 4.920440497768268e+25)